<a href="https://www.kaggle.com/code/ryancardwell/orcaswordv2?scriptVersionId=272248137" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#Cell 1
"""
ARC PRIZE 2025 - ORCASWORD SOLVER v9.0 (META-AWARENESS CORE)
================================================================================
Cell 1/5: Configuration, Core Data Structures, GridOps, and PET Integration.
(Foundation for Meta-Awareness, including multi-dimensional context definition.)
"""

import json
import numpy as np
import time 
from typing import List, Dict, Tuple, Optional, Set, Callable
from dataclasses import dataclass, field
from collections import defaultdict
from itertools import permutations
import copy
from math import sqrt, ceil, floor

# Scipy fallback for Cell 4 functionality
try:
    from scipy.ndimage import label
except ImportError:
    def label(array, structure=None):
        return array, 0

# =============================================================================
# --- Global Configuration & PET Dimensional Constants ---
# =============================================================================

MAX_PERMUTATION_LENGTH = 5      
MAX_VP_SEARCH_ITERATIONS = 5    
MAX_HYPOTHESIS_EXPANSION = 16   
TRAINING_TIME_LIMIT_MIN = 120  # SIGNIFICANTLY INCREASED TRAINING TIME (x3)

# =============================================================================
# --- CORE DATA STRUCTURES & PET CONTEXT --- 
# =============================================================================

Grid = List[List[int]]
TaskTier = str 
PET_KEY = Tuple[str, str, str, str] # (Scale, Dimension, Plane, Axis)

@dataclass
class StrategyMetrics:
    name: str
    cost: int
    orthogonality_score: float = 0.5 # Default, updated in Cell 3/4
    # Standard metrics
    success_count_by_tier: Dict[TaskTier, int] = field(default_factory=lambda: defaultdict(int))
    total_attempts_by_tier: Dict[TaskTier, int] = field(default_factory=lambda: defaultdict(int))
    total_validation_steps: Dict[TaskTier, int] = field(default_factory=lambda: defaultdict(int))
    last_success_time: float = 0.0
    
    # META-AWARENESS CORE: The Primitive Efficacy Tensor (PET)
    # PET: Maps (Tier, Scale, Dim, Plane, Axis) -> [Success_Count, Attempt_Count, Total_NSM_Steps]
    primitive_efficacy_tensor: Dict[PET_KEY, List[int]] = field(default_factory=lambda: defaultdict(lambda: [0, 0, 0]))

@dataclass
class ARCTask:
    task_id: str
    train_examples: List[Tuple[Grid, Grid]]
    test_inputs: List[Grid]

@dataclass
class TaskFeatures:
    grid_size_score: int
    unique_colors: int
    density_ratio: float
    symmetry_score: float
    tier: TaskTier = "Medium"
    # NEW PET Context Features
    scale: str = "Medium"
    dimension: str = "2D"
    plane: str = "XY"
    axis: str = "None"


# =============================================================================
# --- CONSOLIDATED GRID OPERATIONS (Context-Aware Utilities) --- 
# =============================================================================

class GridOps:
    """Consolidated low-level grid manipulation methods."""
    @staticmethod
    def to_numpy(grid: Grid) -> np.ndarray: return np.array(grid, dtype=np.int32)
    @staticmethod
    def from_numpy(arr: np.ndarray) -> Grid: return arr.tolist()
    @staticmethod
    def grids_equal(g1: Grid, g2: Grid) -> bool:
        if not g1 or not g2 or len(g1) != len(g2) or len(g1[0]) != len(g2[0]): return False
        return all(r1 == r2 for r1, r2 in zip(g1, g2))
    @staticmethod
    def rotate_90(grid: Grid, k: int = 1) -> Grid: return GridOps.from_numpy(np.rot90(GridOps.to_numpy(grid), -k))
    @staticmethod
    def reflection_map(grid: Grid, axis: str) -> Grid:
        I = GridOps.to_numpy(grid)
        if axis == 'H': return GridOps.from_numpy(I[:, ::-1])
        if axis == 'V': return GridOps.from_numpy(I[::-1, :])
        return grid
    @staticmethod
    def center_mass_align(grid: Grid) -> Grid:
        I = GridOps.to_numpy(grid); H, W = I.shape; non_zero = np.argwhere(I != 0)
        if non_zero.size == 0: return grid
        center_r, center_c = non_zero.mean(axis=0)
        target_r, target_c = (H - 1) / 2, (W - 1) / 2
        dy, dx = int(round(target_r - center_r)), int(round(target_c - center_c))
        O = np.roll(np.roll(I, dy, axis=0), dx, axis=1)
        if dy > 0: O[:dy, :] = 0;
        if dy < 0: O[H+dy:, :] = 0
        if dx > 0: O[:, :dx] = 0
        if dx < 0: O[:, W+dx:] = 0
        return GridOps.from_numpy(O)
    @staticmethod
    def get_color_mapping(inp: Grid, out: Grid) -> Optional[Dict[int, int]]:
        I_arr, O_arr = GridOps.to_numpy(inp), GridOps.to_numpy(out)
        unique_colors = np.unique(I_arr[I_arr != 0]); color_map = {}
        for c in unique_colors:
            output_colors = np.unique(O_arr[I_arr == c])
            if len(output_colors) == 1:
                color_map[int(c)] = int(output_colors[0])
            elif len(output_colors) > 1: return None 
        if len(color_map.keys()) != len(set(color_map.values())): return None
        return color_map
    @staticmethod
    def replace_color(grid: Grid, old_color: int, new_color: int) -> Grid:
        arr = GridOps.to_numpy(grid)
        arr[arr == old_color] = new_color
        return GridOps.from_numpy(arr)
    @staticmethod
    def boundary_extract(grid: Grid) -> Grid: # Placeholder implementation
        I = GridOps.to_numpy(grid)
        O = np.zeros_like(I)
        H, W = I.shape
        for r in range(1, H - 1):
            for c in range(1, W - 1):
                if I[r, c] != 0 and (I[r-1, c] == 0 or I[r+1, c] == 0 or I[r, c-1] == 0 or I[r, c+1] == 0):
                    O[r, c] = I[r, c]
        return GridOps.from_numpy(O)
        
    # [Context-Aware Feature Extraction]
    @staticmethod
    def calculate_features(task: ARCTask) -> TaskFeatures:
        all_inputs = [GridOps.to_numpy(inp) for inp, _ in task.train_examples]
        if not all_inputs: return TaskFeatures(0, 0, 0.0, 0.0, "Easier")
        
        H, W = len(task.train_examples[0][0]), len(task.train_examples[0][0][0])
        total_cells = H * W * len(task.train_examples)
        
        unique_colors = max(len(np.unique(arr[arr != 0])) for arr in all_inputs)
        non_zero_count = sum(np.sum(arr != 0) for arr in all_inputs)
        density_ratio = non_zero_count / total_cells if total_cells > 0 else 0.0
        grid_size_score = H * W

        sym_score = 0.0
        for arr in all_inputs:
            sym_score += np.mean(arr == arr[:, ::-1]) + np.mean(arr == arr[::-1, :])
        symmetry_score = sym_score / (len(all_inputs) * 2)

        features = TaskFeatures(grid_size_score, unique_colors, density_ratio, symmetry_score)
        
        # --- PET CONTEXT DERIVATION (Meta-Awareness Proof Code Segment) ---
        features.scale = "Small" if H < 15 and W < 15 else "Large" if H > 30 or W > 30 else "Medium"
        features.dimension = "1D" if H == 1 or W == 1 else "2D"
        features.plane = "X-Biased" if W > H * 2 else "Y-Biased" if H > W * 2 else "XY"
        features.axis = "Rotational" if symmetry_score > 0.6 else "Positional"
        
        features.tier = GridOps.determine_tier(features)
        return features
    
    @staticmethod
    def determine_tier(features: TaskFeatures) -> TaskTier:
        complexity_score = (features.unique_colors * 5) + (features.grid_size_score / 10) + (1 - features.symmetry_score) * 20
        
        if complexity_score < 10: return "Easier"
        elif complexity_score < 25: return "Medium"
        elif complexity_score < 40: return "Hard"
        else: return "Elite"

#Cell 1


In [2]:
#Cell 2
"""
ARC PRIZE 2025 - ORCASWORD SOLVER v9.0 (META-AWARENESS CORE)
================================================================================
Cell 2/5: Strategy Hierarchy, Self-Reflective Induction (SRI), and Strategy Factory.
(Implementation of Meta-Orthogonal Search Bias (OSB) and Meta-Awareness of PET context.)
"""

# Must assume imports and classes from Cell 1 are present.

# =============================================================================
# --- STRATEGY FACTORY PATTERN (RETAINED) ---
# =============================================================================

class StrategyRegistry:
    """Centralized dictionary to map strategy names to their class definitions."""
    strategies: Dict[str, Tuple[type, int, float]] = {} # Maps Name -> (Class, Cost, Orthogonality)

    @staticmethod
    def register(name: str, cost: int, orthogonality_score: float = 0.5):
        """Decorator for registering strategies."""
        def decorator(cls):
            if name in StrategyRegistry.strategies:
                raise ValueError(f"Strategy '{name}' already registered.")
            StrategyRegistry.strategies[name] = (cls, cost, orthogonality_score)
            return cls
        return decorator

class StrategyFactory:
    """Instantiates all known strategies."""
    @staticmethod
    def get_all_strategies() -> List['TransformationStrategy']:
        instances = []
        for name, (cls, cost, orthogonality_score) in StrategyRegistry.strategies.items():
            instance = cls()
            instance.cost = cost
            instance.orthogonality_score = orthogonality_score
            instances.append(instance)
        return instances

# =============================================================================
# --- STRATEGY BASE CLASS (RETAINED NSM INTERFACE) ---
# =============================================================================

class TransformationStrategy:
    """Base class defining the standard interface and NSM contract."""
    def __init__(self, name: str, cost: int, orthogonality_score: float = 0.5):
        self.name = name
        self.cost = cost 
        self.orthogonality_score = orthogonality_score
    
    def can_apply_and_cost(self, train_examples: List[Tuple[Grid, Grid]]) -> Tuple[bool, Optional[Dict], int]:
        """Runs standard validation and returns (success, rule_params, steps_taken)."""
        return False, None, 1
    
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        return test_input 

# =============================================================================
# --- SELF-REFLECTIVE INDUCTION (SRI) CORE (META-AWARENESS & OSB) ---
# =============================================================================

class SelfReflectiveInduction:
    """
    SRI Component: Provides the NSM/SDP data by measuring exact effort and
    applying Meta-Orthogonal Search Bias (OSB) using the PET context.
    """
    
    @staticmethod
    def get_nsm_expansion_attempts(strategy_cost: int, task_tier: TaskTier) -> int:
        """Insight 1 (Meta Learning): Dynamically sets the NSM budget."""
        base_attempts = strategy_cost * MAX_HYPOTHESIS_EXPANSION 
        
        if task_tier in ['Hard', 'Elite']:
            return base_attempts * 4
        elif strategy_cost >= 4:
            return base_attempts * 2
        return base_attempts 
    
    @staticmethod
    def validate_and_cost(strategy: 'TransformationStrategy', train_examples: List[Tuple[Grid, Grid]]) -> Tuple[bool, Optional[Dict], int, PET_KEY]:
        """
        Runs validation, measures NSM steps, and applies Meta-Orthogonal Search Bias (OSB).
        Returns: (success, rule_params, steps_taken, pet_context_key)
        """
        features = GridOps.calculate_features(ARCTask(task_id="", train_examples=train_examples, test_inputs=[]))
        task_tier = features.tier
        pet_key: PET_KEY = (features.scale, features.dimension, features.plane, features.axis)

        try:
            success, rule_params, steps_base = strategy.can_apply_and_cost(train_examples)
        except Exception:
             return False, None, 1, pet_key

        if success:
            return True, rule_params, steps_base, pet_key
        
        # 2. Skip deep NSM if cost mismatch is high.
        if strategy.cost < 2 and task_tier in ['Hard', 'Elite']:
            return False, None, steps_base, pet_key
            
        # 3. NSM HYPOTHESIS EXPANSION (Meta Awareness & Orthogonality)
        
        max_nsm_attempts = SelfReflectiveInduction.get_nsm_expansion_attempts(strategy.cost, task_tier)
        nsm_steps_taken = steps_base 
        
        # --- Orthogonal Search Bias (OSB) Implementation ---
        rot_range = [0, 1, 2, 3] 
        if features.axis == 'Positional' or features.plane in ['X-Biased', 'Y-Biased']:
            rot_range = [0, 2] # Restrict rotational search
        
        flip_range = [None, 'H', 'V'] 
        if features.dimension == '1D':
            flip_range = [None, 'H'] # Restrict reflection search
        
        for rot in rot_range:
            for flip in flip_range: 
                
                if nsm_steps_taken >= max_nsm_attempts:
                    return False, None, nsm_steps_taken, pet_key

                perturbed_examples = []
                for inp, out in train_examples:
                    p_inp = GridOps.rotate_90(inp, rot)
                    if flip == 'H': p_inp = GridOps.reflection_map(p_inp, 'H')
                    if flip == 'V': p_inp = GridOps.reflection_map(p_inp, 'V')
                    perturbed_examples.append((p_inp, out))
                
                # Check for identity transformation
                if rot == 0 and flip is None and not GridOps.grids_equal(train_examples[0][0], perturbed_examples[0][0]):
                    # If the base example was transformed despite no rotation/flip, something is wrong.
                    continue

                success, rule_params_inner, steps_inner = strategy.can_apply_and_cost(perturbed_examples)
                nsm_steps_taken += steps_inner

                if success:
                    return True, rule_params_inner, nsm_steps_taken, pet_key

        return False, None, nsm_steps_taken, pet_key 

#Cell 2


In [3]:
#Cell 3 - FIXED VERSION
"""
ARC PRIZE 2025 - ORCASWORD SOLVER v9.0 (META-AWARENESS CORE)
================================================================================
Cell 3/5: Definition of all Cost 1/2/3 strategies, incorporating Primitive Efficacy Tensor (PET) and Orthogonality Score (POS).
(Establishes the Meta-Aware Primitive Hierarchy and models complexity derivatives.)
"""

# NOTE: Assumes imports (numpy, List, Dict, Tuple, Optional, Callable, Set, copy, sqrt, permutations),
# GridOps, StrategyRegistry, ARCTask, TaskTier, and MAX_PERMUTATION_LENGTH are defined in Cell 1/2.

# =============================================================================
# --- PRIMITIVE BASE CLASS (POS INTEGRATION) ---
# =============================================================================

# Redefining for context, assumes consistency with Cell 2/1 structure
class TransformationStrategy: 
    def __init__(self, name: str, cost: int, orthogonality_score: float = 0.5):
        self.name = name
        self.cost = cost 
        self.orthogonality_score = orthogonality_score
    
    # Placeholder definitions - actual logic is in concrete classes
    def can_apply_and_cost(self, train_examples: List[Tuple[Grid, Grid]]) -> Tuple[bool, Optional[Dict], int]:
        return False, None, 1
    
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        return test_input
        
    def _complexity_derivative(self, inp: Grid) -> int:
        """Insight 2 (Meta Awareness): Calculates the operation's computational derivative relative to grid size."""
        # Using math.sqrt and len/indexing for shape; replace with np.sqrt if outside of try/except block.
        H, W = len(inp), len(inp[0])
        return max(1, int(sqrt(H * W) / 2))

# =============================================================================
# --- COST 1 PRIMITIVES (TIER: EASIER) ---
# =============================================================================

@StrategyRegistry.register("identity", 1, orthogonality_score=0.0)
class IdentityStrategy(TransformationStrategy):
    def __init__(self): super().__init__("identity", 1, orthogonality_score=0.0)
    def can_apply_and_cost(self, train_examples): 
        steps = self._complexity_derivative(train_examples[0][0])
        if all(GridOps.grids_equal(inp, out) for inp, out in train_examples): return True, {}, steps
        return False, None, steps
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        return test_input

@StrategyRegistry.register("rotate_90", 1, orthogonality_score=0.8)
class Rotation90Strategy(TransformationStrategy):
    def __init__(self): super().__init__("rotate_90", 1, orthogonality_score=0.8)
    def can_apply_and_cost(self, train_examples): 
        steps = self._complexity_derivative(train_examples[0][0])
        k = 1
        if all(GridOps.grids_equal(GridOps.rotate_90(inp, k), out) for inp, out in train_examples):
            return True, {'k': k}, steps
        return False, None, steps
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        return GridOps.rotate_90(test_input, rule_params.get('k', 1))

@StrategyRegistry.register("rotate_180", 1, orthogonality_score=0.8)
class Rotation180Strategy(TransformationStrategy):
    def __init__(self): super().__init__("rotate_180", 1, orthogonality_score=0.8) 
    def can_apply_and_cost(self, train_examples):
        steps = self._complexity_derivative(train_examples[0][0])
        k = 2
        if all(GridOps.grids_equal(GridOps.rotate_90(inp, k), out) for inp, out in train_examples):
            return True, {'k': k}, steps
        return False, None, steps
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        return GridOps.rotate_90(test_input, rule_params.get('k', 2))

@StrategyRegistry.register("color_map", 1, orthogonality_score=0.2)
class ColorMapStrategy(TransformationStrategy):
    def __init__(self): super().__init__("color_map", 1, orthogonality_score=0.2)
    def can_apply_and_cost(self, train_examples):
        steps = self._complexity_derivative(train_examples[0][0]) * 2
        maps = [GridOps.get_color_mapping(inp, out) for inp, out in train_examples]
        if all(m is not None for m in maps) and all(m == maps[0] for m in maps):
            return True, {'map': maps[0]}, steps
        return False, None, steps
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        if 'map' in rule_params:
            grid = test_input
            for old, new in rule_params['map'].items():
                grid = GridOps.replace_color(grid, old, new)
            return grid
        return test_input
        
# =============================================================================
# --- COST 2 PRIMITIVES (TIER: MEDIUM) ---
# =============================================================================

@StrategyRegistry.register("center_mass_align", 2, orthogonality_score=0.1)
class CenterMassAlignStrategy(TransformationStrategy):
    def __init__(self): super().__init__("center_mass_align", 2, orthogonality_score=0.1)
    def can_apply_and_cost(self, train_examples):
        steps = self._complexity_derivative(train_examples[0][0]) * 3
        # Assuming GridOps.center_mass_align exists and returns a Grid
        if all(GridOps.grids_equal(GridOps.center_mass_align(inp), out) for inp, out in train_examples): return True, {}, steps
        return False, None, steps
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]: return GridOps.center_mass_align(test_input)

@StrategyRegistry.register("boundary_extract", 2, orthogonality_score=0.3)
class BoundaryExtractStrategy(TransformationStrategy):
    def __init__(self): super().__init__("boundary_extract", 2, orthogonality_score=0.3)
    def can_apply_and_cost(self, train_examples):
        steps = self._complexity_derivative(train_examples[0][0]) * 4
        # Assuming GridOps.boundary_extract exists and returns a Grid
        if all(GridOps.grids_equal(GridOps.boundary_extract(inp), out) for inp, out in train_examples): return True, {}, steps
        return False, None, steps
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]: return GridOps.boundary_extract(test_input)

@StrategyRegistry.register("exhaustive_toroidal_shift", 2, orthogonality_score=0.7)
class ExhaustiveToroidalShift(TransformationStrategy):
    def __init__(self): super().__init__("exhaustive_toroidal_shift", 2, orthogonality_score=0.7)
    # Stub: Logic not provided, default to failure
    def can_apply_and_cost(self, train_examples): return False, {}, self._complexity_derivative(train_examples[0][0])
@StrategyRegistry.register("permutation_color_map", 2, orthogonality_score=0.5)
class PermutationColorStrategy(TransformationStrategy):
    def __init__(self): super().__init__("permutation_color_map", 2, orthogonality_score=0.5)
    # Stub: Logic not provided, default to failure
    def can_apply_and_cost(self, train_examples): return False, {}, self._complexity_derivative(train_examples[0][0])

# =============================================================================
# --- COST 3 PRIMITIVES (TIER: HARD) - FUNCTIONAL TIME SINKS ---
# =============================================================================

def variational_erode_op(grid: Grid, T: int) -> Grid:
    """A placeholder for a potentially slow variational operation."""
    I = GridOps.to_numpy(grid)
    for _ in range(T):
        new_I = I.copy(); H, W = I.shape
        # Simplified erosion logic
        for r in range(1, H - 1):
            for c in range(1, W - 1):
                if I[r, c] != 0 and np.any(I[r-1:r+2, c-1:c+2] == 0):
                    new_I[r, c] = 0
        I = new_I
    return GridOps.from_numpy(I)


class VariationalPrimitive(TransformationStrategy):
    """Insight 4 (Meta Interactivity): Base class for strategies that use a dynamic lambda search."""
    def __init__(self, name: str, cost: int, op_func: Callable, base_lambda_range: List[int], orthogonality_score: float):
        super().__init__(name, cost, orthogonality_score)
        self.op_func = op_func
        self.base_lambda_range = base_lambda_range
    
    def _get_lambda_range(self, task_tier: TaskTier) -> List[int]:
        """Dynamic Lambda Range based on Task Tier."""
        # Assuming TaskTier is a class or Enum accessible here
        if task_tier in ['Elite']: return self.base_lambda_range + list(range(max(self.base_lambda_range) + 1, 6))
        return self.base_lambda_range
    
    def _fitness(self, transformed_grid: Grid, output_grid: Grid) -> float:
        I_arr, O_arr = GridOps.to_numpy(transformed_grid), GridOps.to_numpy(output_grid)
        # Simple Mean Squared Error (MSE)
        return np.sum((I_arr - O_arr)**2) / I_arr.size
    
    def can_apply_and_cost(self, train_examples):
        # Assuming GridOps.determine_tier and GridOps.calculate_features are available
        task_tier = GridOps.determine_tier(GridOps.calculate_features(ARCTask(task_id="", train_examples=train_examples, test_inputs=[])))
        lambda_range = self._get_lambda_range(task_tier)
        
        optimal_lambda = None
        steps_taken = 0 
        
        for inp, out in train_examples:
            best_T, best_fit = None, float('inf')
            steps_taken += self._complexity_derivative(inp) * 2
            
            for T in lambda_range: 
                steps_taken += 1
                
                transformed_grid = self.op_func(inp, T)
                fit = self._fitness(transformed_grid, out)
                
                if fit < best_fit:
                    best_fit = fit
                    best_T = T
            
            # Allow a small tolerance for float comparisons
            if best_fit > 0.01: return False, None, steps_taken 

            if optimal_lambda is None: optimal_lambda = best_T
            elif optimal_lambda != best_T: return False, None, steps_taken
                
        return True, {'lambda': optimal_lambda}, steps_taken
    
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        if 'lambda' in rule_params and rule_params['lambda'] is not None:
            return self.op_func(test_input, rule_params['lambda']) 
        return test_input

@StrategyRegistry.register("variational_erosion", 3, orthogonality_score=0.4)
class VariationalErosionStrategy(VariationalPrimitive):
    def __init__(self):
        super().__init__("variational_erosion", 3, variational_erode_op, list(range(1, 4)), orthogonality_score=0.4)


@StrategyRegistry.register("multi_primitive_agent", 3, orthogonality_score=0.9)
class MultiPrimitiveAgent(TransformationStrategy):
    """
    FIXED: Implements non-recursive pool building to resolve the RecursionError.
    """
    def __init__(self): 
        super().__init__("multi_primitive_agent", 3, orthogonality_score=0.9)
        self.primitives_pool = None # FIX 1: Defer initialization to avoid recursion
        self.best_sequence = None 

    def _build_low_cost_pool_non_recursive(self):
        """Helper to build the pool by iterating the Registry directly, avoiding StrategyFactory's recursion."""
        pool = []
        # Filter for strategies with cost <= 2 and excluding the MultiPrimitiveAgent itself
        for name, (cls, cost, orthogonality_score) in StrategyRegistry.strategies.items():
            if cost <= 2 and name != self.name: 
                instance = cls() # Instantiates the specific strategy class
                instance.cost = cost
                instance.orthogonality_score = orthogonality_score
                pool.append(instance)
        return pool

    def can_apply_and_cost(self, train_examples):
        # FIX 2: Initialize pool non-recursively on first call
        if self.primitives_pool is None:
            self.primitives_pool = self._build_low_cost_pool_non_recursive() 

        # If pool is empty (no low-cost primitives registered), fail immediately
        if not self.primitives_pool:
            return False, None, 1

        N = MAX_PERMUTATION_LENGTH
        self.best_sequence = None
        
        primitives_pool_filtered = self.primitives_pool
        # Sorting by orthogonality_score to prioritize diverse sequences first
        primitives_pool_sorted = sorted(primitives_pool_filtered, key=lambda p: p.orthogonality_score, reverse=True)
        
        steps_taken = 0 
        
        # This permutation approach is highly costly and needs MAX_PERMUTATION_LENGTH to be very small (e.g., 2)
        all_sequences_with_pos = []
        # Generate permutations up to length N, considering sequence order and diversity score
        for seq_tuple in permutations(primitives_pool_sorted, N):
            sequence = list(seq_tuple)
            # Calculate cumulative POS (normalized inverse score for diversity)
            cumulative_pos = sum(p.orthogonality_score for p in sequence) / N
            all_sequences_with_pos.append((cumulative_pos, sequence))
            
        all_sequences_with_pos.sort(key=lambda x: x[0], reverse=True) # Check most diverse (highest POS) first

        for cumulative_pos, sequence in all_sequences_with_pos:
            steps_taken += 1 
            sequence_matches = True
            
            steps_taken += sum(p._complexity_derivative(train_examples[0][0]) for p in sequence)
            
            for inp, out in train_examples:
                current_grid = copy.deepcopy(inp)
                for primitive_obj in sequence:
                    # Apply using the primitive's own logic (note: this assumes simple primitives 
                    # don't need rule_params for the combination check here)
                    current_grid = primitive_obj.apply(current_grid, {}) 
                
                if not GridOps.grids_equal(current_grid, out):
                    sequence_matches = False
                    break
            
            if sequence_matches:
                self.best_sequence = sequence
                seq_names = [p.name for p in sequence]
                # Return the sequence names and the steps taken
                return True, {'sequence': seq_names}, steps_taken

        return False, None, steps_taken
    
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        if 'sequence' in rule_params:
            if self.primitives_pool is None:
                self.primitives_pool = self._build_low_cost_pool_non_recursive()
            
            current_grid = copy.deepcopy(test_input)
            sequence_names = rule_params['sequence']
            
            # Map names to instantiated primitive objects in the pool
            name_to_primitive = {p.name: p for p in self.primitives_pool}
            
            for name in sequence_names:
                primitive_obj = name_to_primitive.get(name)
                if primitive_obj:
                    # Apply primitive. Note: if primitives require rule_params, the MultiPrimitiveAgent 
                    # needs to handle parameter search/propagation, which is not implemented here.
                    current_grid = primitive_obj.apply(current_grid, {})
                else: 
                    return test_input # Fail gracefully if a primitive in the sequence is missing
            return current_grid
        return test_input

@StrategyRegistry.register("laplacian_eigenmap", 3, orthogonality_score=0.05)
class LaplacianEigenmapStrategy(TransformationStrategy):
    def __init__(self): super().__init__("laplacian_eigenmap", 3, orthogonality_score=0.05)
    # Stub: Logic not provided, default to failure
    def can_apply_and_cost(self, train_examples): return False, {}, self._complexity_derivative(train_examples[0][0]) * 5

@StrategyRegistry.register("dft_pattern_match", 3, orthogonality_score=0.15)
class DFTMatchStrategy(TransformationStrategy):
    """
    COMPLETELY REWRITTEN: Safe implementation that prevents NoneType errors
    """
    def __init__(self): 
        super().__init__("dft_pattern_match", 3, orthogonality_score=0.15)
        
    def can_apply_and_cost(self, train_examples: List[Tuple[Grid, Grid]]) -> Tuple[bool, Optional[Dict], int]:
        # This strategy is intentionally disabled to prevent errors
        steps = self._complexity_derivative(train_examples[0][0]) * 5
        return False, None, steps

    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        # COMPLETE SAFETY: Always return original input
        return test_input

@StrategyRegistry.register("subgrid_window_perm", 3, orthogonality_score=0.5)
class SubGridWindowPermutationStrategy(TransformationStrategy):
    def __init__(self): super().__init__("subgrid_window_perm", 3, orthogonality_score=0.5)
    # Stub: Logic not provided, default to failure
    def can_apply_and_cost(self, train_examples): return False, {}, self._complexity_derivative(train_examples[0][0]) * 3

#Cell 3

In [4]:
#Cell 4
"""
ARC PRIZE 2025 - ORCASWORD SOLVER v9.0 (META-AWARENESS CORE)
================================================================================
Cell 4/5: NEW Five Novel Synthesis Methods (Cost 4/5 Elite Strategies).
(Implementation of Elite Primitive Validation Depth (EVD), Dynamic Orthogonality Penalization (DOP),
and Multi-Scale Feature Persistence (MFP) Check.)
"""

# Must assume imports and classes from Cell 1, 2, & 3 are present.

# =============================================================================
# --- ELITE STRATEGY BASE & META-AWARENESS HELPERS ---
# =============================================================================

class EliteStrategyHelper:
    """Provides advanced meta-aware methods for Cost 4/5 strategies."""
    
    @staticmethod
    def _get_nsm_validation_depth(strategy: TransformationStrategy, features: 'TaskFeatures') -> int:
        """Elite Primitive Validation Depth (EVD)."""
        base_depth = strategy.cost * 10
        if features.tier == 'Elite':
            return base_depth * 3
        elif features.scale == 'Large':
            return base_depth * 2
        return base_depth

    @staticmethod
    def _dynamic_orthogonality_penalty(strategy: TransformationStrategy, features: 'TaskFeatures') -> int:
        """Dynamic Orthogonality Penalization (DOP)."""
        penalty = 0
        if strategy.cost >= 4 and features.tier in ['Easier', 'Medium']:
            penalty = int(strategy.orthogonality_score * 50)
        return penalty

    @staticmethod
    def check_multi_scale_persistence(inp: Grid, out: Grid) -> bool:
        """Multi-Scale Feature Persistence (MFP) Check."""
        I_arr, O_arr = GridOps.to_numpy(inp), GridOps.to_numpy(out)
        
        # 1. Connected Component Check
        num_components_I, _ = label(I_arr)
        num_components_O, _ = label(O_arr)
        if np.max(num_components_I) != np.max(num_components_O):
            return False

        # 2. Border Pixel Ratio Check
        I_boundary = GridOps.to_numpy(GridOps.boundary_extract(inp))
        O_boundary = GridOps.to_numpy(GridOps.boundary_extract(out))
        
        density_I_boundary = np.sum(I_boundary != 0) / np.sum(I_arr != 0) if np.sum(I_arr != 0) else 0
        density_O_boundary = np.sum(O_boundary != 0) / np.sum(O_arr != 0) if np.sum(O_arr != 0) else 0
        
        if abs(density_I_boundary - density_O_boundary) > 0.1:
            return False
            
        return True


# =============================================================================
# --- NOVEL SYNTHESIS METHODS (COST 4 - ELITE) ---
# =============================================================================

@StrategyRegistry.register("graph_shortest_path", 4, orthogonality_score=0.1)
class GraphShortestPathStrategy(TransformationStrategy):
    def __init__(self): super().__init__("graph_shortest_path", 4, orthogonality_score=0.1)
    
    def can_apply_and_cost(self, train_examples):
        features = GridOps.calculate_features(ARCTask(task_id="", train_examples=train_examples, test_inputs=[]))
        max_steps = EliteStrategyHelper._get_nsm_validation_depth(self, features)
        steps_taken = EliteStrategyHelper._dynamic_orthogonality_penalty(self, features)
        
        for inp, out in train_examples:
            steps_taken += self._complexity_derivative(inp) * 5
            if steps_taken >= max_steps: return False, None, steps_taken
            
            I, O = GridOps.to_numpy(inp), GridOps.to_numpy(out)
            H, W = I.shape
            
            out_colors = {(r, c): O[r, c] for r in range(H) for c in range(W) if O[r, c] != 0}
            if not out_colors: continue

            consistency_map = {}
            for r_in in range(H):
                for c_in in range(W):
                    if I[r_in, c_in] != 0:
                        min_dist_sq = float('inf')
                        nearest_color = None
                        
                        for (r_out, c_out), color in out_colors.items():
                            dist_sq = (r_in - r_out)**2 + (c_in - c_out)**2
                            if dist_sq < min_dist_sq:
                                min_dist_sq = dist_sq
                                nearest_color = color
                        
                        if nearest_color is None: continue

                        if I[r_in, c_in] in consistency_map and consistency_map[I[r_in, c_in]] != nearest_color:
                            return False, None, steps_taken
                        consistency_map[I[r_in, c_in]] = nearest_color
            
        return True, {'map': consistency_map}, steps_taken
    
    def apply(self, test_input, rule_params):
        I = GridOps.to_numpy(test_input); O = np.zeros_like(I)
        if 'map' in rule_params:
            for r in range(I.shape[0]):
                for c in range(I.shape[1]):
                    in_color = I[r, c]
                    if in_color != 0 and in_color in rule_params['map']:
                        O[r, c] = rule_params['map'][in_color]
        return GridOps.from_numpy(O)

@StrategyRegistry.register("segmentation_color_fill", 4, orthogonality_score=0.3)
class SegmentationColorFillStrategy(TransformationStrategy):
    def __init__(self): super().__init__("segmentation_color_fill", 4, orthogonality_score=0.3)
    
    def can_apply_and_cost(self, train_examples):
        features = GridOps.calculate_features(ARCTask(task_id="", train_examples=train_examples, test_inputs=[]))
        max_steps = EliteStrategyHelper._get_nsm_validation_depth(self, features)
        steps_taken = EliteStrategyHelper._dynamic_orthogonality_penalty(self, features)
        
        rule = None
        for inp, out in train_examples:
            steps_taken += self._complexity_derivative(inp) * 4
            if steps_taken >= max_steps: return False, None, steps_taken

            I, O = GridOps.to_numpy(inp), GridOps.to_numpy(out)
            
            input_counts = {c: np.sum(I == c) for c in np.unique(I) if c != 0}
            if not input_counts: return False, None, steps_taken
            target_input_color = max(input_counts, key=input_counts.get)
            
            segment_mask = (I == target_input_color)
            output_colors_in_segment = O[segment_mask]
            
            output_counts = {c: np.sum(output_colors_in_segment == c) for c in np.unique(output_colors_in_segment) if c != 0}
            if not output_counts: continue
            target_output_color = max(output_counts, key=output_counts.get)

            current_rule = (target_input_color, target_output_color)
            
            if rule is None: rule = current_rule
            elif rule != current_rule: return False, None, steps_taken
            
            O_test = O.copy()
            O_test[segment_mask] = target_output_color
            if np.sum(O_test != O) > 0: return False, None, steps_taken
            
        return True, {'in': rule[0], 'out': rule[1]}, steps_taken
    
    def apply(self, test_input, rule_params):
        I = GridOps.to_numpy(test_input); O = I.copy()
        
        target_input_color = rule_params['in']
        target_output_color = rule_params['out']
        
        segment_mask = (I == target_input_color)
        O[segment_mask] = target_output_color
        
        return GridOps.from_numpy(O)

@StrategyRegistry.register("fft_pattern_filter", 5, orthogonality_score=0.0)
class FFTPatternFilterStrategy(TransformationStrategy):
    def __init__(self): super().__init__("fft_pattern_filter", 5, orthogonality_score=0.0)
    
    def can_apply_and_cost(self, train_examples):
        features = GridOps.calculate_features(ARCTask(task_id="", train_examples=train_examples, test_inputs=[]))
        max_steps = EliteStrategyHelper._get_nsm_validation_depth(self, features)
        steps_taken = EliteStrategyHelper._dynamic_orthogonality_penalty(self, features)

        rule = None
        for inp, out in train_examples:
            steps_taken += self._complexity_derivative(inp) * 8
            if steps_taken >= max_steps: return False, None, steps_taken
            
            I = GridOps.to_numpy(inp); O = GridOps.to_numpy(out)
            if I.shape != O.shape: continue

            if not EliteStrategyHelper.check_multi_scale_persistence(inp, out):
                 return False, None, steps_taken

            F_I = np.fft.fft2(I); F_O = np.fft.fft2(O)
            magnitude_I = np.abs(F_I); magnitude_O = np.abs(F_O)
            
            threshold = np.max(magnitude_I) * 0.1
            dominant_mask = (magnitude_I > threshold)

            if np.sum(dominant_mask) == 0: continue
                
            ratios = magnitude_O[dominant_mask] / magnitude_I[dominant_mask]
            
            mean_ratio = np.mean(ratios)
            std_ratio = np.std(ratios)

            if std_ratio > 0.1 * mean_ratio:
                return False, None, steps_taken
            
            current_rule = mean_ratio
            if rule is None: rule = current_rule
            elif abs(rule - current_rule) > 0.1: return False, None, steps_taken
            
        return True, {'mean_ratio': rule}, steps_taken
    
    def apply(self, test_input, rule_params):
        I = GridOps.to_numpy(test_input)
        F_I = np.fft.fft2(I)
        
        F_O = F_I * rule_params['mean_ratio'] 
        
        O = np.fft.ifft2(F_O).real
        O = np.round(np.clip(O, 0, 9)).astype(np.int32) 
        
        return GridOps.from_numpy(O)

# =============================================================================
# --- NOVEL SYNTHESIS METHODS (COST 5 - EXTREME ELITE) ---
# =============================================================================

@StrategyRegistry.register("topological_homology_match", 5, orthogonality_score=0.05)
class TopologicalHomologyMatch(TransformationStrategy):
    def __init__(self): super().__init__("topological_homology_match", 5, orthogonality_score=0.05)
    
    def can_apply_and_cost(self, train_examples):
        features = GridOps.calculate_features(ARCTask(task_id="", train_examples=train_examples, test_inputs=[]))
        max_steps = EliteStrategyHelper._get_nsm_validation_depth(self, features)
        steps_taken = EliteStrategyHelper._dynamic_orthogonality_penalty(self, features)
        
        for inp, out in train_examples:
            steps_taken += self._complexity_derivative(inp) * 10
            if steps_taken >= max_steps: return False, None, steps_taken
            
            I, O = GridOps.to_numpy(inp), GridOps.to_numpy(out)
            
            num_components_I, _ = label(I)
            num_components_O, _ = label(O)
            
            if np.max(num_components_I) != np.max(num_components_O):
                return False, None, steps_taken
                
        return True, {'invariants_preserved': ['component_count']}, steps_taken
    
    def apply(self, test_input, rule_params):
        return test_input 

@StrategyRegistry.register("tensor_decomposition_factor", 5, orthogonality_score=0.1)
class TensorDecompositionFactor(TransformationStrategy):
    def __init__(self): super().__init__("tensor_decomposition_factor", 5, orthogonality_score=0.1)
    
    def can_apply_and_cost(self, train_examples):
        features = GridOps.calculate_features(ARCTask(task_id="", train_examples=train_examples, test_inputs=[]))
        max_steps = EliteStrategyHelper._get_nsm_validation_depth(self, features)
        steps_taken = EliteStrategyHelper._dynamic_orthogonality_penalty(self, features)
        
        rule = None
        
        for inp, out in train_examples:
            steps_taken += self._complexity_derivative(inp) * 12
            if steps_taken >= max_steps: return False, None, steps_taken
            
            I, O = GridOps.to_numpy(inp), GridOps.to_numpy(out)
            
            if I.shape != O.shape: continue 
                
            norm_I = np.linalg.norm(I); norm_O = np.linalg.norm(O)
            
            if norm_I == 0: continue
                
            scale_factor = norm_O / norm_I
            
            if rule is None: rule = scale_factor
            elif abs(rule - scale_factor) > 0.1: return False, None, steps_taken
            
        if rule:
            return True, {'scale': rule}, steps_taken
        return False, None, steps_taken
    
    def apply(self, test_input, rule_params):
        I = GridOps.to_numpy(test_input)
        O = I * rule_params['scale']
        O = np.round(np.clip(O, 0, 9)).astype(np.int32)
        return GridOps.from_numpy(O)

#Cell 4


In [5]:
#Cell 5 - COMPLETE FIXED VERSION
"""
ARC PRIZE 2025 - ORCASWORD SOLVER v9.0 (META-AWARENESS CORE)
================================================================================
Cell 5/5: Meta-Learner (SDP/PET), Solver Core (SRI), and Main Execution Logic.
(Implements Contextual Inductivity Score, Task Refusal, and a 7-Phase Deployment.)
"""

# Must assume imports and classes from Cell 1, 2, 3, & 4 are present.

# =============================================================================
# --- META-LEARNER COMPONENT (SDP/PET ENHANCEMENT) ---
# =============================================================================

class MetaLearner:
    """Core of Meta-Awareness. Manages the Primitive Efficacy Tensor (PET) and determines dynamic strategy order using Contextual Inductivity Score (CIS)."""
    def __init__(self):
        self.all_strategies = StrategyFactory.get_all_strategies()
        # Initialize StrategyMetrics with orthogonality score from the factory
        self.metrics: Dict[str, StrategyMetrics] = {
            s.name: StrategyMetrics(s.name, s.cost, s.orthogonality_score) for s in self.all_strategies
        }
        self.global_solved_task_ids: Set[str] = set()

    # ESSENTIAL PIECE 1: PET Update Mechanism
    def update_metrics(self, strategy_name: str, task_tier: TaskTier, success: bool, validation_steps_taken: int, pet_context_key: PET_KEY):
        """Updates standard metrics and the multi-dimensional Primitive Efficacy Tensor (PET)."""
        metrics = self.metrics[strategy_name]
        
        metrics.total_attempts_by_tier[task_tier] += 1
        metrics.total_validation_steps[task_tier] += validation_steps_taken
        if success:
            metrics.success_count_by_tier[task_tier] += 1
            metrics.last_success_time = time.time()
            
        pet_entry = metrics.primitive_efficacy_tensor[pet_context_key]
        pet_entry[0] += 1 if success else 0
        pet_entry[1] += 1
        pet_entry[2] += validation_steps_taken

    def get_success_rate(self, strategy_name: str, task_tier: TaskTier) -> float:
        metrics = self.metrics[strategy_name]
        attempts = metrics.total_attempts_by_tier[task_tier]
        successes = metrics.success_count_by_tier[task_tier]
        return successes / attempts if attempts > 0 else -1.0 

    # ESSENTIAL PIECE 2: Contextual Inductivity Score (CIS) Calculation
    def calculate_cis(self, strategy_name: str, pet_context_key: PET_KEY) -> float:
        """Calculates the Contextual Inductivity Score (CIS) using PET data."""
        metrics = self.metrics[strategy_name]
        pet_entry = metrics.primitive_efficacy_tensor[pet_context_key]
        
        successes, attempts, total_steps = pet_entry
        orthogonality = metrics.orthogonality_score
        
        if attempts == 0:
            base_inductivity = 1.0 / (metrics.cost + 1e-6)
            return base_inductivity * (1 - orthogonality) * 0.1 

        contextual_success_rate = successes / attempts
        induction_cost = total_steps / attempts
        raw_inductivity = contextual_success_rate / (induction_cost + 1e-6)
        
        # CIS: Higher is better (Success_Rate / Cost * Orthogonality_Bonus)
        cis = raw_inductivity * (1.0 + (1.0 - orthogonality))
        return cis

    # ESSENTIAL PIECE 3: Dynamic Strategy Order with CIS Integration
    def get_dynamic_order(self, features: TaskFeatures, ideology: str) -> List[TransformationStrategy]:
        """Dynamically sorts strategies using Tier-based success, Cost, and Contextual Inductivity (CIS)."""
        task_tier = features.tier
        pet_context_key: PET_KEY = (features.scale, features.dimension, features.plane, features.axis)
        
        def dynamic_sort_key(strategy: TransformationStrategy) -> Tuple[float, float, int]:
            success_rate = self.get_success_rate(strategy.name, task_tier)
            cost_factor = strategy.cost
            cis_score = self.calculate_cis(strategy.name, pet_context_key)
            
            if ideology == 'alpha':
                return (-success_rate, cost_factor, -cis_score)
            else: # Omega (Complex/Breakthrough)
                return (-cis_score, -success_rate, cost_factor) 

        sorted_strategies = sorted(self.all_strategies, key=dynamic_sort_key)
        
        if task_tier in ['Hard', 'Elite'] and ideology == 'omega':
             sorted_strategies.sort(key=lambda s: (-s.cost, -self.calculate_cis(s.name, pet_context_key)))
            
        return sorted_strategies

# =============================================================================
# --- ENHANCED SOLVER CORE (MCAS DUAL-AGENT + SRI) ---
# =============================================================================

class EnhancedARCSolver:
    
    def __init__(self, ideology: str = 'alpha', meta_learner: Optional[MetaLearner] = None):
        self.ideology = ideology
        self.meta_learner = meta_learner
        self.strategies = StrategyFactory.get_all_strategies()
        self.stats = {'total': 0, 'solved': 0, 'strategy_success': defaultdict(int)}
        self.log = []

    # ESSENTIAL PIECE 4: Task Refusal Mechanism (Reliability Check)
    def should_refuse_task(self, features: TaskFeatures) -> bool:
        """Task Refusal for Omega agent when confidence is low."""
        if self.ideology == 'alpha': return False
        
        task_tier = features.tier
        best_rate = -1.0
        best_attempts = 0
        
        for name, metrics in self.meta_learner.metrics.items():
            if metrics.cost >= 3:
                rate = self.meta_learner.get_success_rate(name, task_tier)
                attempts = metrics.total_attempts_by_tier[task_tier]
                
                if rate > best_rate:
                    best_rate = rate
                    best_attempts = attempts
        
        if best_attempts > 50 and best_rate < 0.01:
            return True
        return False

    def solve_task(self, task: ARCTask) -> List[Grid]:
        features = GridOps.calculate_features(task)
        task_tier = features.tier
        
        if self.meta_learner:
            if self.should_refuse_task(features):
                self.log.append(f"REFUSAL|{task.task_id}|{task_tier}")
                return [task.test_inputs[0]]
                
            self.strategies = self.meta_learner.get_dynamic_order(features, self.ideology)
        
        if self.ideology == 'alpha' and self.meta_learner and task.task_id in self.meta_learner.global_solved_task_ids:
            return [copy.deepcopy(t) for t in task.test_inputs] 

        predictions = []
        for test_input in task.test_inputs:
            predictions.append(self._solve_single(test_input, task.train_examples, task.task_id, features))
        return predictions
    
    def _solve_single(self, test_input: Grid, train_examples: List[Tuple[Grid, Grid]], task_id: str, features: TaskFeatures) -> Grid:
        """CRITICAL FIX: Added missing _solve_single method with error handling"""
        self.stats['total'] += 1
        task_tier = features.tier
        pet_context_key: PET_KEY = (features.scale, features.dimension, features.plane, features.axis)
        
        for strategy in self.strategies:
            start_solve = time.time()
            
            try:
                success, rule_params, steps_taken, _ = SelfReflectiveInduction.validate_and_cost(strategy, train_examples)
            except Exception as e:
                self.log.append(f"CRITICAL_FAILURE|{task_id}|{strategy.name}|{task_tier}|{e}")
                success, rule_params, steps_taken = False, None, 1
            
            solve_time = time.time() - start_solve
            
            if success:
                # SAFETY WRAPPER: Prevent strategy application errors
                try:
                    result = strategy.apply(test_input, rule_params)
                except Exception as e:
                    self.log.append(f"CRITICAL_APPLY_FAILURE|{task_id}|{strategy.name}|{task_tier}|{e}")
                    result = test_input
                
                if result is not None and not GridOps.grids_equal(result, test_input):
                    self.stats['solved'] += 1
                    self.stats['strategy_success'][strategy.name] += 1
                    
                    if self.meta_learner:
                        self.meta_learner.update_metrics(strategy.name, task_tier, True, steps_taken, pet_context_key)
                    
                    # ESSENTIAL PIECE 6: Enhanced SDM Observability Log
                    self.log.append(f"SUCCESS|{task_id}|{strategy.name}|{task_tier}|{steps_taken}|{solve_time:.4f}|{strategy.cost}|PET:{':'.join(pet_context_key)}|CIS:{self.meta_learner.calculate_cis(strategy.name, pet_context_key):.4f}")
                    
                    return result
            
            if self.meta_learner:
                self.meta_learner.update_metrics(strategy.name, task_tier, False, steps_taken, pet_context_key)
            
            # ESSENTIAL PIECE 6: Enhanced SDM Observability Log
            self.log.append(f"FAILURE|{task_id}|{strategy.name}|{task_tier}|{steps_taken}|{solve_time:.4f}|{strategy.cost}|PET:{':'.join(pet_context_key)}")

        return test_input

# =============================================================================
# --- DATA LOADING & SUBMISSION UTILITIES (RETAINED) ---
# =============================================================================

class ARCDataLoader:
    @staticmethod
    def load_tasks(challenges_path: str) -> List[ARCTask]:
        try:
            with open(challenges_path, 'r') as f: challenges = json.load(f)
        except FileNotFoundError: return []
        
        tasks = []
        for task_id, task_data in challenges.items():
            train_examples = [(ex['input'], ex['output']) for ex in task_data['train']]
            test_inputs = [ex['input'] for ex in task_data['test']]
            tasks.append(ARCTask(task_id=task_id, train_examples=train_examples, test_inputs=test_inputs))
        return tasks

class SubmissionGenerator:
    # ESSENTIAL PIECE 7: Robust Submission Generation
    @staticmethod
    def generate(predictions: Dict[str, List[Grid]], output_path: str):
        submission = {}
        for task_id, pred_grids in predictions.items():
            sanitized_grids = []
            if pred_grids:
                first_pred = pred_grids[0]
                if isinstance(first_pred, list):
                     sanitized_grids.append(first_pred)
            
            final_pred = sanitized_grids[0] if sanitized_grids else [[0]] 
            
            submission[task_id] = {
                "attempt_1": final_pred,
                "attempt_2": final_pred 
            }
        
        if output_path:
            with open(output_path, 'w') as f: 
                json.dump(submission, f, indent=2)

# =============================================================================
# --- MAIN EXECUTION (SEVEN-PHASE NOVEL SYNTHESIS DEPLOYMENT) ---
# =============================================================================

def merge_predictions(current_predictions: Dict, new_predictions: Dict, meta_learner: MetaLearner) -> Dict:
    # Merging logic (Retained for robustness)
    newly_solved_ids = set()
    for task_id, new_preds in new_predictions.items():
        if not new_preds: continue
        new_grid = new_preds[0]
        is_new_solution = new_grid and any(any(cell != 0 for cell in row) for row in new_grid) 

        if task_id not in current_predictions:
            current_predictions[task_id] = new_preds
            if is_new_solution: newly_solved_ids.add(task_id) 
        else:
            current_grid = current_predictions[task_id][0]
            is_current_solution = current_grid and any(any(cell != 0 for cell in row) for row in current_grid)
            
            if is_new_solution and not is_current_solution:
                 current_predictions[task_id] = new_preds
                 newly_solved_ids.add(task_id)
            
    meta_learner.global_solved_task_ids.update(newly_solved_ids)
    return current_predictions

# ESSENTIAL PIECE 5: Meta-Aware Retraining Loop
def run_prediction_phase(
    solver: EnhancedARCSolver, 
    tasks: List[ARCTask], 
    time_budget_minutes: int, 
    phase_name: str,
    reverse_order: bool = False
):
    time_budget_seconds = time_budget_minutes * 60
    order_label = "REVERSED" if reverse_order else "NORMAL"
    print(f"\n--- {phase_name.upper()} | AGENT {solver.ideology.upper()} | {order_label} ORDER ({time_budget_minutes} MIN BUDGET) ---")
    
    solver.stats = {'total': 0, 'solved': 0, 'strategy_success': defaultdict(int)}
    solver.log = []
    
    tasks_to_run = tasks[::-1] if reverse_order else tasks
    
    start_time = time.time()
    predictions = {}
    tasks_attempted = 0
    
    for i, task in enumerate(tasks_to_run):
        elapsed_time = time.time() - start_time
        
        if elapsed_time >= time_budget_seconds:
            print(f"   Time limit reached! Stopping at task {i}/{len(tasks_to_run)}.")
            break
            
        if i > 0 and i % 50 == 0:
            tasks_per_second = i / elapsed_time if elapsed_time > 0 else 0
            print(f"   Progress: {i}/{len(tasks_to_run)} | Elapsed: {elapsed_time:.0f}s | Speed: {tasks_per_second:.2f} tasks/sec")
            
        predictions[task.task_id] = solver.solve_task(task)
        tasks_attempted += 1

    final_elapsed = time.time() - start_time
    
    print(f"✅ {phase_name.upper()} COMPLETE!")
    print(f"   Tasks Attempted: {tasks_attempted} / {len(tasks_to_run)}")
    print(f"   Total Solutions Found: {solver.stats['solved']} / {solver.stats['total']} attempts")
    print(f"   Final Elapsed Time: {final_elapsed:.2f}s")
    
    # SDM Feature: Elite Strategy Performance Summary
    print("\n   Elite Strategy Performance (Cost >= 4):")
    for s_name in sorted(solver.stats['strategy_success'].keys()):
        if StrategyRegistry.strategies[s_name][1] >= 4 and solver.stats['strategy_success'][s_name] > 0:
            print(f"      {s_name:<30}: {solver.stats['strategy_success'][s_name]} solutions")
    print("-" * 80)
    
    return predictions


def main():
    print("="*80)
    print("ARC PRIZE 2025 - ORCASWORD SOLVER v9.0 (NSM/SDP - FINAL META-CORE ARCHITECTURE)")
    print("="*80)
    
    PHASE_BUDGETS = {
        'P0_TRAIN': TRAINING_TIME_LIMIT_MIN,
        'P1_ALPHA_TEST': 10,
        'P2_OMEGA_EVAL': 10,
        'P3_ALPHA_EVAL': 5,
        'P4_OMEGA_TEST': 5,
        'P5_CONSOL_TEST': 2,
        'P6_CONSOL_EVAL': 2,
    }
    
    KAGGLE_INPUT_DIR = '/kaggle/input/arc-prize-2025'
    KAGGLE_OUTPUT_DIR = '/kaggle/working'
    
    TRAIN_PATH = f'{KAGGLE_INPUT_DIR}/arc-agi_training_challenges.json'
    TEST_PATH = f'{KAGGLE_INPUT_DIR}/arc-agi_test_challenges.json'          
    EVAL_PATH = f'{KAGGLE_INPUT_DIR}/arc-agi_evaluation_challenges.json'  
    
    TEST_OUTPUT_PATH = f'{KAGGLE_OUTPUT_DIR}/submission_test.json'
    EVAL_OUTPUT_PATH = f'{KAGGLE_OUTPUT_DIR}/submission_eval.json'
    
    print(f"\n📚 Loading data...")
    train_tasks = ARCDataLoader.load_tasks(TRAIN_PATH)
    test_tasks = ARCDataLoader.load_tasks(TEST_PATH)
    eval_tasks = ARCDataLoader.load_tasks(EVAL_PATH)
    print(f"   Loaded Train: {len(train_tasks)} | Test: {len(test_tasks)} | Eval: {len(eval_tasks)} tasks.")

    if not test_tasks and not eval_tasks: 
        print("No test or evaluation tasks loaded. Exiting.")
        return

    meta_learner = MetaLearner()

    # --- PHASE 0: META-TRAINING (PET & SDP Seeding) ---
    print("\n\n--- PHASE 0: META-TRAINING (NSM/SDP PET Seeding) ---")
    
    meta_train_agent = EnhancedARCSolver(ideology='omega', meta_learner=meta_learner)
    
    p0_preds = run_prediction_phase(
        solver=meta_train_agent, tasks=train_tasks, time_budget_minutes=PHASE_BUDGETS['P0_TRAIN'], 
        phase_name="TRAINING CHALLENGES (P0 - Deep PET Seed)", reverse_order=False
    )
    
    print("\n🧠 MetaLearner Knowledge Snapshot (Top Contextual Inductivity Scores):")
    sample_key = ('Large', '2D', 'XY', 'Rotational')
    print(f"   Target PET Context: {sample_key}")
    
    cis_list = []
    for s_name in meta_learner.metrics.keys():
        cis = meta_learner.calculate_cis(s_name, sample_key)
        if meta_learner.metrics[s_name].total_attempts_by_tier.get('Elite', 0) > 0 and cis > 0.01:
             cis_list.append((s_name, cis))
    
    for s_name, cis in sorted(cis_list, key=lambda x: x[1], reverse=True)[:5]:
        print(f"      {s_name:<30} | CIS: {cis:.4f}")


    # --- PHASE 1-6: SEVEN-PHASE DYNAMIC DEPLOYMENT ---

    final_test_predictions = {}
    final_eval_predictions = {}
    
    alpha_agent = EnhancedARCSolver(ideology='alpha', meta_learner=meta_learner)
    omega_agent = EnhancedARCSolver(ideology='omega', meta_learner=meta_learner)
    
    # P1: Initial Coverage (Alpha on Test)
    test_preds_run1 = run_prediction_phase(alpha_agent, test_tasks, PHASE_BUDGETS['P1_ALPHA_TEST'], "TEST (P1 - Initial Alpha)", reverse_order=False)
    final_test_predictions = merge_predictions(final_test_predictions, test_preds_run1, meta_learner)
    
    # P2: Complex Search (Omega on Eval, Reversed for anti-sequential bias)
    eval_preds_run2 = run_prediction_phase(omega_agent, eval_tasks, PHASE_BUDGETS['P2_OMEGA_EVAL'], "EVAL (P2 - Omega Breakthrough)", reverse_order=True)
    final_eval_predictions = merge_predictions(final_eval_predictions, eval_preds_run2, meta_learner)
    
    # P3: Alpha Cross-Validation (Alpha on Eval)
    eval_preds_run3 = run_prediction_phase(alpha_agent, eval_tasks, PHASE_BUDGETS['P3_ALPHA_EVAL'], "EVAL (P3 - Alpha Cross-Val)", reverse_order=False)
    final_eval_predictions = merge_predictions(final_eval_predictions, eval_preds_run3, meta_learner)

    # P4: Omega Cross-Validation (Omega on Test, Reversed)
    test_preds_run4 = run_prediction_phase(omega_agent, test_tasks, PHASE_BUDGETS['P4_OMEGA_TEST'], "TEST (P4 - Omega Breakthrough)", reverse_order=True)
    final_test_predictions = merge_predictions(final_test_predictions, test_preds_run4, meta_learner)

    # P5: Consolidation Sweep 1 (Alpha on Test, Final Low-Hanging Fruit)
    test_preds_run5 = run_prediction_phase(alpha_agent, test_tasks, PHASE_BUDGETS['P5_CONSOL_TEST'], "TEST (P5 - Consolidation Sweep)", reverse_order=False)
    final_test_predictions = merge_predictions(final_test_predictions, test_preds_run5, meta_learner)

    # P6: Consolidation Sweep 2 (Omega on Eval, Final Complex Search)
    eval_preds_run6 = run_prediction_phase(omega_agent, eval_tasks, PHASE_BUDGETS['P6_CONSOL_EVAL'], "EVAL (P6 - Final Complex Search)", reverse_order=False)
    final_eval_predictions = merge_predictions(final_eval_predictions, eval_preds_run6, meta_learner)


    # --- PHASE 7: FINAL SUBMISSION GENERATION (Implicit Phase) ---
    print("\n" + "="*80)
    print("🔥 AGGREGATING AND GENERATING FINAL SUBMISSIONS (Phase 7: Submission)")
    
    SubmissionGenerator.generate(final_test_predictions, TEST_OUTPUT_PATH)
    SubmissionGenerator.generate(final_eval_predictions, EVAL_OUTPUT_PATH)
    
    print("\n✅ Execution Complete. Final submissions written.")
    print(f"Total Unique Test Tasks Solved (Consolidated): {len(final_test_predictions)}")
    print(f"Total Unique Eval Tasks Solved (Consolidated): {len(final_eval_predictions)}")
    print("="*80)

# Explicitly call main() for robust execution
if __name__ == '__main__':
    main()
#Cell 5

ARC PRIZE 2025 - ORCASWORD SOLVER v9.0 (NSM/SDP - FINAL META-CORE ARCHITECTURE)

📚 Loading data...
   Loaded Train: 1000 | Test: 240 | Eval: 120 tasks.


--- PHASE 0: META-TRAINING (NSM/SDP PET Seeding) ---

--- TRAINING CHALLENGES (P0 - DEEP PET SEED) | AGENT OMEGA | NORMAL ORDER (120 MIN BUDGET) ---
   Progress: 50/1000 | Elapsed: 154s | Speed: 0.32 tasks/sec
   Progress: 100/1000 | Elapsed: 382s | Speed: 0.26 tasks/sec
   Progress: 150/1000 | Elapsed: 597s | Speed: 0.25 tasks/sec
   Progress: 200/1000 | Elapsed: 834s | Speed: 0.24 tasks/sec
   Progress: 250/1000 | Elapsed: 1029s | Speed: 0.24 tasks/sec
   Progress: 300/1000 | Elapsed: 1246s | Speed: 0.24 tasks/sec
   Progress: 350/1000 | Elapsed: 1484s | Speed: 0.24 tasks/sec
   Progress: 400/1000 | Elapsed: 1651s | Speed: 0.24 tasks/sec
   Progress: 450/1000 | Elapsed: 1853s | Speed: 0.24 tasks/sec
   Progress: 500/1000 | Elapsed: 2102s | Speed: 0.24 tasks/sec
   Progress: 550/1000 | Elapsed: 2294s | Speed: 0.24 tasks/sec
   Progre